In [ ]:
from fastai.vision.all import *
from fastai.callback.all import *
from fasterai.core.criteria import *
import torch_pruning as tp
from torch_pruning.pruner import function
import torch_pruning as tp
import timm

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def onecycle_scheduler(pruning_ratio_dict, steps, start=0, end=1, α=14, β=6):
    return [
        sched_onecycle(start, end, i / float(steps), α, β) * pruning_ratio_dict
        for i in range(steps + 1)
    ]

def sched_onecycle(start, end, pos, α=14, β=6):
    out = (1 + np.exp(-α + β)) / (1 + np.exp((-α * pos) + β))
    return start + (end - start) * out

In [ ]:
def get_dls(size, bs):
    path = URLs.IMAGENETTE_160
    source = untar_data(path)
    blocks=(ImageBlock, CategoryBlock)
    tfms = [RandomResizedCrop(size, min_scale=0.35), FlipItem(0.5)]
    batch_tfms = [Normalize.from_stats(*imagenet_stats)]

    csv_file = 'noisy_imagenette.csv'
    inp = pd.read_csv(source/csv_file)
    dblock = DataBlock(blocks=blocks,
               splitter=ColSplitter(),
               get_x=ColReader('path', pref=source),
               get_y=ColReader(f'noisy_labels_0'),
               item_tfms=tfms,
               batch_tfms=batch_tfms)

    return dblock.dataloaders(inp, path=source, bs=bs)

In [ ]:
model = timm.create_model('resnet18', pretrained=False, no_jit=True).eval()

In [ ]:
dls = get_dls(model.default_cfg['input_size'][2], 16)

In [ ]:
#learn = vision_learner(dls, 'bat_resnext26ts', metrics = [accuracy])
#learn.unfreeze()

model = timm.create_model('beit_base_patch16_224', pretrained=False, no_jit=True).eval()

ignored_layers = []
num_heads = {}
pruning_ratio_dict = {}
#ratios = [0.265625,0.234375,0.265625,0.265625,0.93359375,0.328125,0.2265625,0.58984375,0.54296875,0.701171875,0.919921875,0.04296875,0.796875,0.240966796875,0.07763671875]


#k = 0
for m in model.modules():
    #if hasattr(m, 'head'): #isinstance(m, nn.Linear) and m.out_features == model.num_classes:
    if isinstance(m, nn.Linear) and m.out_features == model.num_classes:
        ignored_layers.append(m)
        print("Ignore classifier layer: ", m)

    # Attention layers
    if hasattr(m, 'num_heads'):
        if hasattr(m, 'qkv'):
            num_heads[m.qkv] = m.num_heads
            print("Attention layer: ", m.qkv, m.num_heads)
        elif hasattr(m, 'qkv_proj'):
            num_heads[m.qkv_proj] = m.num_heads
    
    #elif isinstance(m, nn.Conv2d):
    #    pruning_ratio_dict[m] = ratios[k]
    #    print(k)
    #    k+=1

learn = Learner(dls, model, metrics = [accuracy])

Attention layer:  Linear(in_features=768, out_features=2304, bias=False) 12
Attention layer:  Linear(in_features=768, out_features=2304, bias=False) 12
Attention layer:  Linear(in_features=768, out_features=2304, bias=False) 12
Attention layer:  Linear(in_features=768, out_features=2304, bias=False) 12
Attention layer:  Linear(in_features=768, out_features=2304, bias=False) 12
Attention layer:  Linear(in_features=768, out_features=2304, bias=False) 12
Attention layer:  Linear(in_features=768, out_features=2304, bias=False) 12
Attention layer:  Linear(in_features=768, out_features=2304, bias=False) 12
Attention layer:  Linear(in_features=768, out_features=2304, bias=False) 12
Attention layer:  Linear(in_features=768, out_features=2304, bias=False) 12
Attention layer:  Linear(in_features=768, out_features=2304, bias=False) 12
Attention layer:  Linear(in_features=768, out_features=2304, bias=False) 12
Ignore classifier layer:  Linear(in_features=768, out_features=1000, bias=True)


In [ ]:
xb, _ = dls.one_batch()

NameError: name 'dls' is not defined

In [ ]:
learn.fit_one_cycle(3, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.302582,1.146651,0.622166,00:44
1,0.953016,0.809723,0.741147,00:45
2,0.828404,0.692629,0.775796,00:44


In [ ]:
pruner = tp.pruner.MetaPruner(
                        model, 
                        xb.to('cpu'), 
                        global_pruning=False,
                        importance=tp.importance.GroupNormImportance(), 
                        iterative_steps=10000,
                        pruning_ratio=0.5,
                        #pruning_ratio_dict=pruning_ratio_dict,
                        num_heads=num_heads,
                        ignored_layers=ignored_layers,
                    )
#for g in pruner.step(interactive=True):
#    g.prune()
pruner.step()

/home/HubensN/miniconda3/envs/fasterai/lib/python3.9/site-packages/torch_pruning/dependency.py:697: UserWarning: Unwrapped parameters detected: ['cls_token', 'blocks.1.attn.v_bias', 'blocks.6.attn.relative_position_bias_table', 'blocks.0.attn.v_bias', 'blocks.3.gamma_2', 'blocks.3.attn.q_bias', 'blocks.4.attn.relative_position_bias_table', 'blocks.5.attn.relative_position_bias_table', 'blocks.9.gamma_2', 'blocks.9.attn.q_bias', 'blocks.11.attn.relative_position_bias_table', 'blocks.2.attn.relative_position_bias_table', 'blocks.4.gamma_2', 'blocks.6.attn.q_bias', 'blocks.7.attn.q_bias', 'blocks.7.attn.relative_position_bias_table', 'blocks.9.attn.v_bias', 'blocks.10.gamma_2', 'blocks.10.attn.relative_position_bias_table', 'blocks.0.attn.q_bias', 'blocks.0.attn.relative_position_bias_table', 'blocks.1.gamma_2', 'blocks.4.attn.q_bias', 'blocks.8.attn.q_bias', 'blocks.8.attn.v_bias', 'blocks.10.attn.q_bias', 'blocks.11.gamma_1', 'blocks.11.gamma_2', 'blocks.0.gamma_2', 'blocks.5.gamma_1', 

In [ ]:
for m in model.modules():
    # Attention layers
    if hasattr(m, 'num_heads'):
        if hasattr(m, 'qkv'):
            m.num_heads = num_heads[m.qkv]
            m.head_dim = m.qkv.out_features // (3 * m.num_heads)
        elif hasattr(m, 'qkv_proj'):
            m.num_heads = num_heads[m.qqkv_projkv]
            m.head_dim = m.qkv_proj.out_features // (3 * m.num_heads)

In [ ]:
model_old = timm.create_model('convnext_xxlarge', pretrained=False, no_jit=True).eval()
base_macs, base_params = tp.utils.count_ops_and_params(model_old, xb.to('cpu'))

In [ ]:
pruned_macs, pruned_params = tp.utils.count_ops_and_params(model, xb.to('cpu'))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (3152x767 and 768x2304)

In [ ]:
print("MACs: %.4f G => %.4f G"%(base_macs/1e9, pruned_macs/1e9))
print("Params: %.4f M => %.4f M"%(base_params/1e6, pruned_params/1e6))

MACs: 151.5881 G => 7.0919 G
Params: 846.4710 M => 58.2010 M


In [ ]:
class PruneCallback(Callback):
    def __init__(self, pruning_ratio, schedule, criteria, ignored_layers, *args, **kwargs):
        store_attr()
        self.sparsity_levels = []
        self.extra_args = args
        self.extra_kwargs = kwargs

    def before_fit(self):
        n_batches_per_epoch = len(self.learn.dls.train)
        total_training_steps = n_batches_per_epoch * self.learn.n_epoch

        self.total_training_steps = total_training_steps 
        print(self.total_training_steps)
        self.example_inputs, _ = self.learn.dls.one_batch()
        self.sparsity_levels = self.schedule(self.pruning_ratio, total_training_steps)

        self.pruner = tp.pruner.MetaPruner(
        self.learn.model,
        example_inputs= torch.randn(self.example_inputs.shape).to('cuda:0'),
        importance=self.criteria,
        pruning_ratio=self.pruning_ratio, 
        ignored_layers=self.ignored_layers,
        iterative_steps= self.total_training_steps, 
        #iterative_steps= 1, 
        #iterative_pruning_ratio_scheduler=self.schedule,
        #global_pruning=self.context, 
        *self.extra_args, 
        **self.extra_kwargs
        )
        
    def before_step(self):
        if self.training: 
           #self.pruner.step()
            for g in self.pruner.step(interactive=True):
                g.prune()
            
        #for m in self.pruner.model.modules():
        #    # Attention layers
        #    if hasattr(m, 'num_heads'):
        #        if hasattr(m, 'qkv'):
        #            m.num_heads = num_heads[m.qkv]
        #            m.head_dim = m.qkv.out_features // (3 * m.num_heads)
        #        elif hasattr(m, 'qkv_proj'):
        #            m.num_heads = num_heads[m.qqkv_projkv]
        #            m.head_dim = m.qkv_proj.out_features // (3 * m.num_heads)

    def after_epoch(self):
        completed_steps = (self.epoch + 1) * len(self.learn.dls.train)
        current_sparsity = self.sparsity_levels[completed_steps - 1]
        print(f'Sparsity at the end of epoch {self.epoch}: {current_sparsity*100:.2f}%')

In [ ]:
timm.list_models()

In [ ]:
model = timm.create_model('resnet18', pretrained=True, no_jit=True).eval()

ignored_layers = []
num_heads = {}

#k = 0
for m in model.modules():
    if isinstance(m, nn.Linear) and m.out_features == model.num_classes:
        ignored_layers.append(m)
        print("Ignore classifier layer: ", m)

    # Attention layers
    if hasattr(m, 'num_heads'):
        if hasattr(m, 'qkv'):
            num_heads[m.qkv] = m.num_heads
            print("Attention layer: ", m.qkv, m.num_heads)
        elif hasattr(m, 'qkv_proj'):
            num_heads[m.qkv_proj] = m.num_heads

learn = Learner(dls, model, metrics = [accuracy])

Ignore classifier layer:  Linear(in_features=512, out_features=1000, bias=True)


In [ ]:
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.410828,0.291440,0.907771,00:12
1,0.378970,0.237411,0.920764,00:12
2,0.254743,0.161710,0.946752,00:12
3,0.195507,0.142778,0.954395,00:12
4,0.124310,0.112485,0.965350,00:12


In [ ]:
pr_cb = PruneCallback(pruning_ratio=0.25, schedule=onecycle_scheduler, global_pruning=True, criteria=tp.importance.GroupNormImportance(normalizer=None, target_types=[nn.modules.conv._ConvNd, nn.Linear]), num_heads=num_heads, ignored_layers=ignored_layers)
learn.fit_one_cycle(10, 1e-4, cbs=pr_cb)

5910


epoch,train_loss,valid_loss,accuracy,time
0,0.190778,0.223286,0.929172,00:16
1,0.906825,0.996437,0.685605,00:14
2,2.154549,2.038115,0.295796,00:14
3,2.019212,1.988954,0.316688,00:13


Sparsity at the end of epoch 0: 0.25%
Sparsity at the end of epoch 1: 0.98%
Sparsity at the end of epoch 2: 3.54%
Sparsity at the end of epoch 3: 10.02%



KeyboardInterrupt



In [ ]:
pr_cb = PruneCallback(pruning_ratio=0.25, schedule=onecycle_scheduler, global_pruning=True, criteria=GroupNormImportance(normalizer=None, target_types=[nn.modules.conv._ConvNd, nn.Linear]), num_heads=num_heads, ignored_layers=ignored_layers)
learn.fit_one_cycle(15, 1e-4, cbs=pr_cb)

8865


epoch,train_loss,valid_loss,accuracy,time
0,0.118503,0.118367,0.963057,00:16
1,0.128137,0.115662,0.962038,00:16
2,0.122357,0.126387,0.960255,00:16
3,0.142834,0.112623,0.966115,00:16
4,0.124087,0.115061,0.964076,00:16
5,0.134429,0.125728,0.960510,00:16
6,0.128918,0.143516,0.954650,00:16
7,0.131373,0.176553,0.958217,00:16
8,0.135654,0.177501,0.954395,00:16
9,0.180335,0.197476,0.946497,00:16


Sparsity at the end of epoch 0: 0.16%
Sparsity at the end of epoch 1: 0.39%
Sparsity at the end of epoch 2: 0.98%
Sparsity at the end of epoch 3: 2.35%
Sparsity at the end of epoch 4: 5.21%
Sparsity at the end of epoch 5: 10.03%
Sparsity at the end of epoch 6: 15.75%
Sparsity at the end of epoch 7: 20.31%
Sparsity at the end of epoch 8: 22.93%
Sparsity at the end of epoch 9: 24.15%
Sparsity at the end of epoch 10: 24.66%
Sparsity at the end of epoch 11: 24.87%
Sparsity at the end of epoch 12: 24.95%
Sparsity at the end of epoch 13: 24.99%
Sparsity at the end of epoch 14: 25.00%


In [ ]:
model = timm.create_model('tf_efficientnet_b3', pretrained=False, no_jit=True).eval()
base_macs, base_params = tp.utils.count_ops_and_params(model, xb.to('cpu'))

In [ ]:
pruned_macs, pruned_params = tp.utils.count_ops_and_params(learn.model, xb.to('cuda:0'))

In [ ]:
print("MACs: %.4f G => %.4f G"%(base_macs/1e9, pruned_macs/1e9))
print("Params: %.4f M => %.4f M"%(base_params/1e6, pruned_params/1e6))

MACs: 0.9399 G => 0.5407 G
Params: 12.2332 M => 7.2501 M


In [ ]:
import abc
import torch
import torch.nn as nn

import typing

from torch_pruning import function
from torch_pruning.dependency import Group

class Importance(abc.ABC):
    """ Estimate the importance of a tp.Dependency.Group, and return an 1-D per-channel importance score.

        It should accept a group as inputs, and return a 1-D tensor with the same length as the number of channels.
        All groups must be pruned simultaneously and thus their importance should be accumulated across channel groups.

        Example:
            ```python
            DG = tp.DependencyGraph().build_dependency(model, example_inputs=torch.randn(1,3,224,224)) 
            group = DG.get_pruning_group( model.conv1, tp.prune_conv_out_channels, idxs=[2, 6, 9] )    
            scorer = MagnitudeImportance()    
            imp_score = scorer(group)    
            #imp_score is a 1-D tensor with length 3 for channels [2, 6, 9]  
            min_score = imp_score.min() 
            ``` 
    """
    @abc.abstractclassmethod
    def __call__(self, group: Group) -> torch.Tensor: 
        raise NotImplementedError


class GroupNormImportance(Importance):

    def __init__(self, 
                 p: int=2, 
                 group_reduction: str="mean", 
                 normalizer: str='mean', 
                 bias=False,
                 target_types:list=[nn.modules.conv._ConvNd, nn.Linear, nn.modules.batchnorm._BatchNorm, nn.LayerNorm]):
        self.p = p
        self.group_reduction = group_reduction
        self.normalizer = normalizer
        self.target_types = target_types
        self.bias = bias

    def _lamp(self, scores): # Layer-adaptive Sparsity for the Magnitude-based Pruning
        """
        Normalizing scheme for LAMP.
        """
        # sort scores in an ascending order
        sorted_scores,sorted_idx = scores.view(-1).sort(descending=False)
        # compute cumulative sum
        scores_cumsum_temp = sorted_scores.cumsum(dim=0)
        scores_cumsum = torch.zeros(scores_cumsum_temp.shape,device=scores.device)
        scores_cumsum[1:] = scores_cumsum_temp[:len(scores_cumsum_temp)-1]
        # normalize by cumulative sum
        sorted_scores /= (scores.sum() - scores_cumsum)
        # tidy up and output
        new_scores = torch.zeros(scores_cumsum.shape,device=scores.device)
        new_scores[sorted_idx] = sorted_scores
        
        return new_scores.view(scores.shape)
    
    def _normalize(self, group_importance, normalizer):
        if normalizer is None:
            return group_importance
        elif isinstance(normalizer, typing.Callable):
            return normalizer(group_importance)
        elif normalizer == "sum":
            return group_importance / group_importance.sum()
        elif normalizer == "standarization":
            return (group_importance - group_importance.min()) / (group_importance.max() - group_importance.min()+1e-8)
        elif normalizer == "mean":
            return group_importance / group_importance.mean()
        elif normalizer == "max":
            return group_importance / group_importance.max()
        elif normalizer == 'gaussian':
            return (group_importance - group_importance.mean()) / (group_importance.std()+1e-8)
        elif normalizer.startswith('sentinel'): # normalize the score with the k-th smallest element. e.g. sentinel_0.5 means median normalization
            sentinel = float(normalizer.split('_')[1]) * len(group_importance)
            sentinel = torch.argsort(group_importance, dim=0, descending=False)[int(sentinel)]
            return group_importance / (group_importance[sentinel]+1e-8)
        elif normalizer=='lamp':
            return self._lamp(group_importance)
        else:
            raise NotImplementedError

    def _reduce(self, group_imp: typing.List[torch.Tensor], group_idxs: typing.List[typing.List[int]]):
        if len(group_imp) == 0: return group_imp
        if self.group_reduction == 'prod':
            reduced_imp = torch.ones_like(group_imp[0])
        elif self.group_reduction == 'max':
            reduced_imp = torch.ones_like(group_imp[0]) * -99999
        else:
            reduced_imp = torch.zeros_like(group_imp[0])

        for i, (imp, root_idxs) in enumerate(zip(group_imp, group_idxs)):
            imp = imp.to(reduced_imp.device)
            if self.group_reduction == "sum" or self.group_reduction == "mean":
                reduced_imp.scatter_add_(0, torch.tensor(root_idxs, device=imp.device), imp) # accumulated importance
            elif self.group_reduction == "max": # keep the max importance
                selected_imp = torch.index_select(reduced_imp, 0, torch.tensor(root_idxs, device=imp.device))
                selected_imp = torch.maximum(input=selected_imp, other=imp)
                reduced_imp.scatter_(0, torch.tensor(root_idxs, device=imp.device), selected_imp)
            elif self.group_reduction == "prod": # product of importance
                selected_imp = torch.index_select(reduced_imp, 0, torch.tensor(root_idxs, device=imp.device))
                torch.mul(selected_imp, imp, out=selected_imp)
                reduced_imp.scatter_(0, torch.tensor(root_idxs, device=imp.device), selected_imp)
            elif self.group_reduction == 'first':
                if i == 0:
                    reduced_imp.scatter_(0, torch.tensor(root_idxs, device=imp.device), imp)
            elif self.group_reduction == 'gate':
                if i == len(group_imp)-1:
                    reduced_imp.scatter_(0, torch.tensor(root_idxs, device=imp.device), imp)
            elif self.group_reduction is None:
                reduced_imp = torch.stack(group_imp, dim=0) # no reduction
            else:
                raise NotImplementedError
        
        if self.group_reduction == "mean":
            reduced_imp /= len(group_imp)
        return reduced_imp
    
    @torch.no_grad()
    def __call__(self, group: Group):
        group_imp = []
        group_idxs = []
        # Iterate over all groups and estimate group importance
        for i, (dep, idxs) in enumerate(group):
            layer = dep.layer
            prune_fn = dep.pruning_fn
            root_idxs = group[i].root_idxs
            if not isinstance(layer, tuple(self.target_types)):
                continue
            ####################
            # Conv/Linear Output
            ####################
            if prune_fn in [
                function.prune_conv_out_channels,
                function.prune_linear_out_channels,
            ]:
                if hasattr(layer, "transposed") and layer.transposed:
                    w = layer.weight.data.transpose(1, 0)[idxs].flatten(1)
                else:
                    w = layer.weight.data[idxs].flatten(1)
                #local_imp = w.abs().pow(self.p).sum(1)
                local_imp = w.abs().pow(self.p).mean(1)
                group_imp.append(local_imp)
                group_idxs.append(root_idxs)

                if self.bias and layer.bias is not None:
                    local_imp = layer.bias.data[idxs].abs().pow(self.p)
                    group_imp.append(local_imp)
                    group_idxs.append(root_idxs)

            ####################
            # Conv/Linear Input
            ####################
            elif prune_fn in [
                function.prune_conv_in_channels,
                function.prune_linear_in_channels,
            ]:
                if hasattr(layer, "transposed") and layer.transposed:
                    w = (layer.weight.data).flatten(1)
                else:
                    w = (layer.weight.data).transpose(0, 1).flatten(1)
                #local_imp = w.abs().pow(self.p).sum(1)
                local_imp = w.abs().pow(self.p).mean(1)

                # repeat importance for group convolutions
                if prune_fn == function.prune_conv_in_channels and layer.groups != layer.in_channels and layer.groups != 1:
                    local_imp = local_imp.repeat(layer.groups)
                
                local_imp = local_imp[idxs]
                group_imp.append(local_imp)
                group_idxs.append(root_idxs)

            ####################
            # BatchNorm
            ####################
            elif prune_fn == function.prune_batchnorm_out_channels:
                # regularize BN
                if layer.affine:
                    w = layer.weight.data[idxs]
                    local_imp = w.abs().pow(self.p)
                    group_imp.append(local_imp)
                    group_idxs.append(root_idxs)

                    if self.bias and layer.bias is not None:
                        local_imp = layer.bias.data[idxs].abs().pow(self.p)
                        group_imp.append(local_imp)
                        group_idxs.append(root_idxs)
            ####################
            # LayerNorm
            ####################
            elif prune_fn == function.prune_layernorm_out_channels:

                if layer.elementwise_affine:
                    w = layer.weight.data[idxs]
                    local_imp = w.abs().pow(self.p)
                    group_imp.append(local_imp)
                    group_idxs.append(root_idxs)

                    if self.bias and layer.bias is not None:
                        local_imp = layer.bias.data[idxs].abs().pow(self.p)
                        group_imp.append(local_imp)
                        group_idxs.append(root_idxs)

        if len(group_imp) == 0: # skip groups without parameterized layers
            return None

        group_imp = self._reduce(group_imp, group_idxs)
        group_imp = self._normalize(group_imp, self.normalizer)
        return group_imp


In [ ]:
timm_models = timm.list_models(module='resnet')

In [ ]:
timm.list_models(exclude_filters='*vit*')

['bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'beitv2_base_patch16_224',
 'beitv2_large_patch16_224',
 'botnet26t_256',
 'botnet50ts_256',
 'caformer_b36',
 'caformer_m36',
 'caformer_s18',
 'caformer_s36',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_medium',
 'coat_lite_medium_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_small',
 'coat_tiny',
 'coatnet_0_224',
 'coatnet_0_rw_224',
 'coatnet_1_224',
 'coatnet_1_rw_224',
 'coatnet_2_224',
 'coatnet_2_rw_224',
 'coatnet_3_224',
 'coatnet_3_rw_224',
 'coatnet_4_224',
 'coatnet_5_224',
 'coatnet_bn_0_rw_224',
 'coatnet_nano_cc_224',
 'coatnet_nano_rw_224',
 'coatnet_pico_rw_224',
 'coatnet_rmlp_0_rw_224',
 'coatnet_rmlp_1_rw2_224',
 'coatnet_rmlp_1_r

In [ ]:
m = timm.create_model('seresnet18')

In [ ]:
m

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (se): SEModule(
        (fc1): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1))
        (bn): Identity()
        (act): ReLU(inplace=True)
        (fc2): Conv2d(8, 64, kernel_size=(1, 1), 

In [ ]:
from fasterai.prune.all import *

In [ ]:
dummy_input = torch.randn(16, 3, 224, 224)

In [ ]:
m = timm.create_model('seresnet18')

In [ ]:
benchmark(m, dummy_input)

Inference time CPU (ms/image):6.694 ms +/- 0.092 ms
FPS CPU: 149.3904126039974
Inference time GPU (ms/image): 0.267 ms +/- 0.003 ms
FPS GPU: 3743.6464639069623
Nombre de paramètres: 11.780 M
Taille du modèle: 44.936 MiB
Nombre de MACs: 1817.400 M


(107.10191986960126, 1.479711068861181, 4.27390784740448, 0.05194234267945458)

In [ ]:
pr = Pruner(m, 'local', large_final, layer_type=[nn.Conv2d])
pr.prune_model(30, round_to=8)

In [ ]:
benchmark(m, dummy_input)

Inference time CPU (ms/image):3.316 ms +/- 0.040 ms
FPS CPU: 301.55608187204695
Inference time GPU (ms/image): 0.183 ms +/- 0.003 ms
FPS GPU: 5469.600455484674
Nombre de paramètres: 5.657 M
Taille du modèle: 21.580 MiB
Nombre de MACs: 836.963 M


(53.058124050003244,
 0.6349589479868257,
 2.9252593731880188,
 0.04034922064140752)

In [ ]:
import abc
import torch
import torch.nn as nn

import typing

from torch_pruning import function
from torch_pruning.dependency import Group

class Importance(abc.ABC):
    """ Estimate the importance of a tp.Dependency.Group, and return an 1-D per-channel importance score.

        It should accept a group as inputs, and return a 1-D tensor with the same length as the number of channels.
        All groups must be pruned simultaneously and thus their importance should be accumulated across channel groups.

        Example:
            ```python
            DG = tp.DependencyGraph().build_dependency(model, example_inputs=torch.randn(1,3,224,224)) 
            group = DG.get_pruning_group( model.conv1, tp.prune_conv_out_channels, idxs=[2, 6, 9] )    
            scorer = MagnitudeImportance()    
            imp_score = scorer(group)    
            #imp_score is a 1-D tensor with length 3 for channels [2, 6, 9]  
            min_score = imp_score.min() 
            ``` 
    """
    @abc.abstractclassmethod
    def __call__(self, group: Group) -> torch.Tensor: 
        raise NotImplementedError


class GroupNormImportance(Importance):

    def __init__(self, 
                 p: int=2, 
                 group_reduction: str="mean", 
                 normalizer: str='mean', 
                 bias=False,
                 target_types:list=[nn.modules.conv._ConvNd, nn.Linear, nn.modules.batchnorm._BatchNorm, nn.LayerNorm]):
        self.p = p
        self.group_reduction = group_reduction
        self.normalizer = normalizer
        self.target_types = target_types
        self.bias = bias

    def _lamp(self, scores): # Layer-adaptive Sparsity for the Magnitude-based Pruning
        """
        Normalizing scheme for LAMP.
        """
        # sort scores in an ascending order
        sorted_scores,sorted_idx = scores.view(-1).sort(descending=False)
        # compute cumulative sum
        scores_cumsum_temp = sorted_scores.cumsum(dim=0)
        scores_cumsum = torch.zeros(scores_cumsum_temp.shape,device=scores.device)
        scores_cumsum[1:] = scores_cumsum_temp[:len(scores_cumsum_temp)-1]
        # normalize by cumulative sum
        sorted_scores /= (scores.sum() - scores_cumsum)
        # tidy up and output
        new_scores = torch.zeros(scores_cumsum.shape,device=scores.device)
        new_scores[sorted_idx] = sorted_scores
        
        return new_scores.view(scores.shape)
    
    def _normalize(self, group_importance, normalizer):
        if normalizer is None:
            return group_importance
        elif isinstance(normalizer, typing.Callable):
            return normalizer(group_importance)
        elif normalizer == "sum":
            return group_importance / group_importance.sum()
        elif normalizer == "standarization":
            return (group_importance - group_importance.min()) / (group_importance.max() - group_importance.min()+1e-8)
        elif normalizer == "mean":
            return group_importance / group_importance.mean()
        elif normalizer == "max":
            return group_importance / group_importance.max()
        elif normalizer == 'gaussian':
            return (group_importance - group_importance.mean()) / (group_importance.std()+1e-8)
        elif normalizer.startswith('sentinel'): # normalize the score with the k-th smallest element. e.g. sentinel_0.5 means median normalization
            sentinel = float(normalizer.split('_')[1]) * len(group_importance)
            sentinel = torch.argsort(group_importance, dim=0, descending=False)[int(sentinel)]
            return group_importance / (group_importance[sentinel]+1e-8)
        elif normalizer=='lamp':
            return self._lamp(group_importance)
        else:
            raise NotImplementedError

    def _reduce(self, group_imp: typing.List[torch.Tensor], group_idxs: typing.List[typing.List[int]]):
        if len(group_imp) == 0: return group_imp
        if self.group_reduction == 'prod':
            reduced_imp = torch.ones_like(group_imp[0])
        elif self.group_reduction == 'max':
            reduced_imp = torch.ones_like(group_imp[0]) * -99999
        else:
            reduced_imp = torch.zeros_like(group_imp[0])

        for i, (imp, root_idxs) in enumerate(zip(group_imp, group_idxs)):
            imp = imp.to(reduced_imp.device)
            if self.group_reduction == "sum" or self.group_reduction == "mean":
                reduced_imp.scatter_add_(0, torch.tensor(root_idxs, device=imp.device), imp) # accumulated importance
            elif self.group_reduction == "max": # keep the max importance
                selected_imp = torch.index_select(reduced_imp, 0, torch.tensor(root_idxs, device=imp.device))
                selected_imp = torch.maximum(input=selected_imp, other=imp)
                reduced_imp.scatter_(0, torch.tensor(root_idxs, device=imp.device), selected_imp)
            elif self.group_reduction == "prod": # product of importance
                selected_imp = torch.index_select(reduced_imp, 0, torch.tensor(root_idxs, device=imp.device))
                torch.mul(selected_imp, imp, out=selected_imp)
                reduced_imp.scatter_(0, torch.tensor(root_idxs, device=imp.device), selected_imp)
            elif self.group_reduction == 'first':
                if i == 0:
                    reduced_imp.scatter_(0, torch.tensor(root_idxs, device=imp.device), imp)
            elif self.group_reduction == 'gate':
                if i == len(group_imp)-1:
                    reduced_imp.scatter_(0, torch.tensor(root_idxs, device=imp.device), imp)
            elif self.group_reduction is None:
                reduced_imp = torch.stack(group_imp, dim=0) # no reduction
            else:
                raise NotImplementedError
        
        if self.group_reduction == "mean":
            reduced_imp /= len(group_imp)
        return reduced_imp
    
    @torch.no_grad()
    def __call__(self, group: Group):
        group_imp = []
        group_idxs = []
        # Iterate over all groups and estimate group importance
        for i, (dep, idxs) in enumerate(group):
            layer = dep.layer
            prune_fn = dep.pruning_fn
            root_idxs = group[i].root_idxs
            if not isinstance(layer, tuple(self.target_types)):
                continue
            ####################
            # Conv/Linear Output
            ####################
            if prune_fn in [
                function.prune_conv_out_channels,
                function.prune_linear_out_channels,
            ]:
                if hasattr(layer, "transposed") and layer.transposed:
                    w = layer.weight.data.transpose(1, 0)[idxs].flatten(1)
                else:
                    w = layer.weight.data[idxs].flatten(1)
                #local_imp = w.abs().pow(self.p).sum(1)
                local_imp = w.abs().pow(self.p).mean(1)
                group_imp.append(local_imp)
                group_idxs.append(root_idxs)

                if self.bias and layer.bias is not None:
                    local_imp = layer.bias.data[idxs].abs().pow(self.p)
                    group_imp.append(local_imp)
                    group_idxs.append(root_idxs)

            ####################
            # Conv/Linear Input
            ####################
            elif prune_fn in [
                function.prune_conv_in_channels,
                function.prune_linear_in_channels,
            ]:
                if hasattr(layer, "transposed") and layer.transposed:
                    w = (layer.weight.data).flatten(1)
                else:
                    w = (layer.weight.data).transpose(0, 1).flatten(1)
                #local_imp = w.abs().pow(self.p).sum(1)
                local_imp = w.abs().pow(self.p).mean(1)

                # repeat importance for group convolutions
                if prune_fn == function.prune_conv_in_channels and layer.groups != layer.in_channels and layer.groups != 1:
                    local_imp = local_imp.repeat(layer.groups)
                
                local_imp = local_imp[idxs]
                group_imp.append(local_imp)
                group_idxs.append(root_idxs)

            ####################
            # BatchNorm
            ####################
            elif prune_fn == function.prune_batchnorm_out_channels:
                # regularize BN
                if layer.affine:
                    w = layer.weight.data[idxs]
                    local_imp = w.abs().pow(self.p)
                    group_imp.append(local_imp)
                    group_idxs.append(root_idxs)

                    if self.bias and layer.bias is not None:
                        local_imp = layer.bias.data[idxs].abs().pow(self.p)
                        group_imp.append(local_imp)
                        group_idxs.append(root_idxs)
            ####################
            # LayerNorm
            ####################
            elif prune_fn == function.prune_layernorm_out_channels:

                if layer.elementwise_affine:
                    w = layer.weight.data[idxs]
                    local_imp = w.abs().pow(self.p)
                    group_imp.append(local_imp)
                    group_idxs.append(root_idxs)

                    if self.bias and layer.bias is not None:
                        local_imp = layer.bias.data[idxs].abs().pow(self.p)
                        group_imp.append(local_imp)
                        group_idxs.append(root_idxs)

        if len(group_imp) == 0: # skip groups without parameterized layers
            return None

        group_imp = self._reduce(group_imp, group_idxs)
        group_imp = self._normalize(group_imp, self.normalizer)
        return group_imp
